# 実行環境の作成

## GCP認証

下記コードでGCPに接続

In [1]:
from google.colab import auth
auth.authenticate_user()

認証に成功したらgcsfuseをインストール

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get -y -q update
!apt-get -y -q install gcsfuse

バケット「statistics-hyogo」をディレクトリ「statistics-hyogo」にマウント

In [ ]:
! mkdir -p statistics-japan
! gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 statistics-japan statistics-japan

## ライブラリ

In [246]:
import pandas as pd
import plotly.express as px
# Google Colab.でプロットするためには，以下を実行する．
import plotly.io as pio
pio.renderers.default = "colab"

import json
import tqdm
import os

# GBQからデータ取得

## テーブルデータ取得

In [5]:
from google.cloud import bigquery

def get_estat_origin(table_id):
  project_id='primal-buttress-342908'
  dataset_id = 'estat_ranking'
  client = bigquery.Client(project=project_id)

  query_string = f'SELECT * FROM {dataset_id}.{table_id}'

  return client.query(query_string).result().to_dataframe()

In [6]:
table_id = '0000010101_A1101'
get_estat_origin(table_id).head()

,index,categoryCode,areaCode,timeCode,value,unit,categoryName,areaName,timeName,rankJapan
0,45,A1101,01000,1975100000,5338206,人,総人口,北海道,1975年度,5
1,46,A1101,01000,1976100000,5395000,人,総人口,北海道,1976年度,5
2,47,A1101,01000,1977100000,5443000,人,総人口,北海道,1977年度,5
3,48,A1101,01000,1978100000,5490000,人,総人口,北海道,1978年度,5
4,49,A1101,01000,1979100000,5535000,人,総人口,北海道,1979年度,5


## テーブルデータを結合

In [214]:
card = {
    'cardId': 'total-population',
    'categories': [
        { '総数' : '0000010101_A1101' },
        { '男性' : '0000010101_A110101' },
        { '女性' : '0000010101_A110102' },
    ]
}

In [50]:
import pandas as pd

def df_data(card):
  
  # 返却するDataFrameの定義
  df_res = pd.DataFrame(index=[], columns=[])

  # 結合
  for c in card['categories']:
    df = get_estat_origin(list(c.values())[0])
    df['categoryName'] = list(c.keys())[0]
    df['timeCode'] = df.apply(lambda x: x['timeCode'][:4], 1)
    df['rankJapan'] = df.astype(str).apply(lambda x: '全国 第' + x['rankJapan'] + '位', 1)
    df = df.drop('index', axis=1)

    df_res = pd.concat([df_res, df])
  
  return df_res

In [44]:
df_data(card).head()

,categoryCode,areaCode,timeCode,value,unit,categoryName,areaName,timeName,rankJapan
0,A1101,01000,1975,5338206,人,総数,北海道,1975年度,全国 第5位
1,A1101,01000,1976,5395000,人,総数,北海道,1976年度,全国 第5位
2,A1101,01000,1977,5443000,人,総数,北海道,1977年度,全国 第5位
3,A1101,01000,1978,5490000,人,総数,北海道,1978年度,全国 第5位
4,A1101,01000,1979,5535000,人,総数,北海道,1979年度,全国 第5位


# 人口・世帯

## Line & Column

In [257]:
"""
apexchartsのLineChartをJSONに保存する関数（地域ごと）

"""

def save_apexcharts_column(cardId,df):

  areas = sorted(list(set(df['areaCode'].tolist())))
  for area in tqdm.notebook.tqdm(areas):
    
    # 地域で抽出
    df_area = df.copy().query(f'areaCode == "{area}"').reset_index()

    # レスポンスを整形
    dic = dict_apexcharts_column(df_area)
 
    # GCSに保存
    gcs_dir = "statistics-japan/apexcharts/{}".format(cardId)
    os.makedirs(gcs_dir, exist_ok=True)
    with open(f'{gcs_dir}/{area}.json', 'w') as f:
      json.dump(dic,f,ensure_ascii=False)
    
  return 


def dict_apexcharts_column(df):
  return {
        'catgories': apexcharts_column_categories(df),
        'series': apexcharts_column_series(df),
    }

def apexcharts_column_categories(df):
  return df['timeCode'].drop_duplicates().reset_index()['timeCode'].tolist()

def apexcharts_column_series(df):
  res = []

  categories = df['categoryName'].drop_duplicates().reset_index()['categoryName'].tolist()
  for categoryName in list_categories(df):
    df_cat = df[df.copy()['categoryName'] == categoryName]
    res.append({
        'name': categoryName, 
        'data': apexcharts_column_data(df_cat),
        'rank': apexcharts_column_rank(df_cat),
    })
  return res

def apexcharts_column_data(df):
  return df.astype(({"value": object}))['value'].tolist()

def apexcharts_column_rank(df):
  return df.astype(({"rankJapan": object}))['rankJapan'].tolist()

#### 総人口

In [244]:
card = {
    'cardId': 'total-population',
    'categories': [
        { '総数' : '0000010101_A1101' },
        { '男性' : '0000010101_A110101' },
        { '女性' : '0000010101_A110102' },
    ]
}
df = df_data(card)

In [258]:
save_apexcharts_column(card['cardId'],df)

  0%|          | 0/47 [00:00<?, ?it/s]

### 年齢調整死亡率

In [259]:
card = {
    'cardId': 'mortality',
    'categories': [
        { '男性' : '0000010101_A424001' },
        { '女性' : '0000010101_A424002' },
    ]
}
df = df_data(card)

In [260]:
save_apexcharts_column(card['cardId'],df)

  0%|          | 0/47 [00:00<?, ?it/s]

### 年齢中位数

In [271]:
card = {
    'cardId': 'median-age',
    'categories': [
        { '年齢中位数' : '0000010101_A1231' },
    ]
}
df = df_data(card)

In [272]:
save_apexcharts_column(card['cardId'],df)

  0%|          | 0/47 [00:00<?, ?it/s]

### 年齢３区分人口

In [274]:
card = {
    'cardId': 'age-population',
    'categories': [
        { '15歳未満人口' : '0000010101_A1301' },
        { '15～64歳人口' : '0000010101_A1302' },
        { '65歳以上人口' : '0000010101_A1303' },
    ]
}
	
df = df_data(card)
df.head()

,categoryCode,areaCode,timeCode,value,unit,categoryName,areaName,timeName,rankJapan
0,A1301,01000,1975,1312611,人,15歳未満人口,北海道,1975年度,全国 第6位
1,A1301,01000,1976,1309000,人,15歳未満人口,北海道,1976年度,全国 第6位
2,A1301,01000,1977,1302000,人,15歳未満人口,北海道,1977年度,全国 第6位
3,A1301,01000,1978,1293000,人,15歳未満人口,北海道,1978年度,全国 第6位
4,A1301,01000,1979,1283000,人,15歳未満人口,北海道,1979年度,全国 第6位


In [275]:
save_apexcharts_column(card['cardId'],df)

  0%|          | 0/47 [00:00<?, ?it/s]

## Pyramid

### 年齢別人口

In [264]:
"""
apexchartsのPyramidChartをJSONに保存する関数（地域ごと）

"""
def save_apexcharts_pyramid(cardId,df):
  
  areas = sorted(list(set(df['areaCode'].tolist())))
  for area in tqdm.notebook.tqdm(areas):

    # 地域のデータを抽出
    df_area = df.copy().query(f'areaCode == "{area}"').reset_index()
    
    # レスポンスを整形
    dic = dict_area(df_area)
    
    # GCSに保存
    gcs_dir = "statistics-japan/apexcharts/{}".format(cardId)
    os.makedirs(gcs_dir, exist_ok=True)
    with open(f'{gcs_dir}/{area}.json', 'w') as f:
      json.dump(dic,f,ensure_ascii=False)

  return

def dict_area(df):
  return {
        'catgories': list_categories(df),
        'times': list_times(df),
        'series': time_series(df),
    }

def time_series(df):
  res = []
  for time in list_times(df):
    df_time = df[df.copy()['timeCode'] == time['timeCode']]
    res.append({
        'timeCode': time['timeCode'],
        'series': [
            {'name': '男性', 'data': list_man(df_time)},
            {'name': '女性', 'data': list_woman(df_time)},
        ]
    })

  return res

def list_man(df):
  df = df.query('categoryName.str.contains("男")', engine='python').astype(({"value": object}))
  return df['value'].tolist()

def list_woman(df):
  df = df.query('categoryName.str.contains("女")', engine='python').astype(({"value": object}))
  return list(map(lambda x: x*-1, df['value'].tolist()))

def list_categories(df):
  return df['categoryName'].str.replace('（女）', '').str.replace('（男）', '').drop_duplicates().reset_index()['categoryName'].tolist()

def list_times(df):
  return df[['timeCode','timeName']].drop_duplicates(subset=['timeCode']).sort_values('timeCode').to_dict(orient='records')


In [265]:
card = {
    'cardId': 'median-age',
    'categories': [
        { "0～4歳人口（男）": "0000010101_A120101"}, 
        { "0～4歳人口（女）": "0000010101_A120102"}, 
        { "5～9歳人口（男）": "0000010101_A120201"}, 
        { "5～9歳人口（女）": "0000010101_A120202"}, 
        { "10～14歳人口（男）": "0000010101_A120301"}, 
        { "10～14歳人口（女）": "0000010101_A120302"}, 
        { "15～19歳人口（男）": "0000010101_A120401"}, 
        { "15～19歳人口（女）": "0000010101_A120402"},  
        { "20～24歳人口（男）": "0000010101_A120501"}, 
        { "20～24歳人口（女）": "0000010101_A120502"}, 
        { "25～29歳人口（男）": "0000010101_A120601"}, 
        { "25～29歳人口（女）": "0000010101_A120602"}, 
        { "30～34歳人口（男）": "0000010101_A120701"}, 
        { "30～34歳人口（女）": "0000010101_A120702"}, 
        { "35～39歳人口（男）": "0000010101_A120801"}, 
        { "35～39歳人口（女）": "0000010101_A120802"}, 
        { "40～44歳人口（男）": "0000010101_A120901"}, 
        { "40～44歳人口（女）": "0000010101_A120902"}, 
        { "45～49歳人口（男）": "0000010101_A121001"}, 
        { "45～49歳人口（女）": "0000010101_A121002"}, 
        { "50～54歳人口（男）": "0000010101_A121101"}, 
        { "50～54歳人口（女）": "0000010101_A121102"}, 
        { "55～59歳人口（男）": "0000010101_A121201"}, 
        { "55～59歳人口（女）": "0000010101_A121202"}, 
        { "60～64歳人口（男）": "0000010101_A121301"}, 
        { "60～64歳人口（女）": "0000010101_A121302"}, 
        { "65～69歳人口（男）": "0000010101_A121401"}, 
        { "65～69歳人口（女）": "0000010101_A121402"}, 
        { "70～74歳人口（男）": "0000010101_A121501"}, 
        { "70～74歳人口（女）": "0000010101_A121502"}, 
        { "75～79歳人口（男）": "0000010101_A121601"}, 
        { "75～79歳人口（女）": "0000010101_A121602"}, 
        { "80～84歳人口（男）": "0000010101_A121701"}, 
        { "80～84歳人口（女）": "0000010101_A121702"}, 
        { "85～89歳人口（男）": "0000010101_A121801"}, 
        { "85～89歳人口（女）": "0000010101_A121802"}, 
        { "90～94歳人口（男）": "0000010101_A121901"}, 
        { "90～94歳人口（女）": "0000010101_A121902"}, 
        { "95～99歳人口（男）": "0000010101_A122001"}, 
        { "95～99歳人口（女）": "0000010101_A122002"}, 
        { "100歳以上人口（男）": "0000010101_A122101"}, 
        { "100歳以上人口（女）": "0000010101_A122102"}
    ]
}
df = df_data(card)

In [267]:
save_apexcharts_pyramid(card['cardId'],df)

  0%|          | 0/47 [00:00<?, ?it/s]